<a href="https://colab.research.google.com/github/JoaoMMS98/ML_2.0/blob/main/Model_Selection_baseline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Model Selection

In this notebook, we are going to apply different ways to split your dataset and try to select the best model.

__`Step 1`__ Import the needed libraries

In [1]:
!pip install xgboost==1.7.5
!pip install scikit-learn==1.2.2
!pip install catboost==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.3
    Uninstalling xgboost-2.1.3:
      Successfully uninstalled xgboost-2.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 102.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 26.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Chiel > XGBoost,CatBoost.

Mariana > KNN, RFM, SVM

In [3]:
# Import the data
train_df = pd.read_csv('traindf.csv')
val_df = pd.read_csv('valdf.csv')
test_df = pd.read_csv('testdf.csv')

In [4]:
train_df

,Claim Injury Type,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,...,Agreement Reached,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M
0,4. TEMPORARY,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,PROPERTY AND CASUALTY,0.497548,QUEENS,...,0.0,1.497677,2021.0,1,1,1,1,0,0,0
1,3. MED ONLY,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,HEALTH & HOSPITAL CORP.,0.212179,BRONX,...,0.0,-0.501860,2022.0,1,1,1,0,0,0,0
2,2. NON-COMP,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,AMERICAN ZURICH INSURANCE CO,0.497548,KINGS,...,0.0,-0.001976,2020.0,1,1,1,0,0,0,1
3,2. NON-COMP,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,"NORDSTROM, INC.",0.091843,KINGS,...,0.0,-1.001744,2021.0,1,1,1,0,0,0,1
4,3. MED ONLY,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,NEW YORK BLACK CAR OPERATORS',0.091843,NASSAU,...,0.0,0.497909,2022.0,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,4. TEMPORARY,-1.241742,-0.150000,1.409856,1.245247,-0.162524,-2.651452,INDEMNITY INSURANCE CO OF,0.497548,WESTCHESTER,...,0.0,1.497677,2021.0,1,1,1,1,0,0,1
459216,4. TEMPORARY,0.884725,-0.009259,1.383011,-0.858729,-0.017208,-2.307054,REDWOOD FIRE & CASUALTY,0.497548,QUEENS,...,0.0,-1.001744,2021.0,1,1,1,1,0,0,1
459217,4. TEMPORARY,1.031378,-0.292593,1.361083,-1.003831,-0.309751,0.000000,UTICA MUTUAL INS CO,0.497548,ERIE,...,0.0,0.997793,2021.0,1,1,1,0,0,0,0
459218,2. NON-COMP,-1.241742,0.642593,0.000000,1.172696,0.655832,0.000000,NEW HAMPSHIRE INSURANCE CO,0.497548,KINGS,...,0.0,-1.001744,2020.0,1,1,1,0,0,0,1


In [5]:
val_df

,Claim Injury Type,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,...,Agreement Reached,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M
0,2. NON-COMP,0.664746,0.122222,0.000000,-0.641076,0.118547,-1.991701,STATE INSURANCE FUND,0.193546,KINGS,...,0.0,-1.501629,2021.0,1,1,1,1,0,0,0
1,2. NON-COMP,-1.095089,-0.648148,0.000000,1.172696,-0.676864,0.000000,STATE INSURANCE FUND,0.193546,KINGS,...,0.0,-1.501629,2022.0,1,1,1,0,0,0,0
2,4. TEMPORARY,-1.021763,-0.685185,1.329181,1.100146,-0.715105,0.000000,PENNSYLVANIA MANUFACTURERS',0.497548,ORANGE,...,0.0,-1.001744,2022.0,1,1,1,1,0,0,0
3,2. NON-COMP,0.664746,-0.446296,0.000000,-0.568525,-0.468451,0.000000,AIU INSURANCE CO,0.497548,ROCKLAND,...,0.0,-0.501860,2022.0,1,1,1,0,0,0,1
4,4. TEMPORARY,0.811399,0.792593,1.330622,-0.858729,0.810707,0.000000,"ALBANY, COUNTY OF",0.212179,ALBANY,...,0.0,-0.001976,2020.0,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114800,4. TEMPORARY,-0.215172,0.120370,1.376163,0.229535,0.116635,0.000000,PUBLIC EMPLOYERS RISK MGMT.,0.212179,DELAWARE,...,0.0,-0.001976,2021.0,1,1,1,0,0,0,1
114801,2. NON-COMP,-0.361825,0.068519,0.000000,0.374636,0.059273,-2.045643,STATE INSURANCE FUND,0.193546,QUEENS,...,0.0,-1.501629,2021.0,1,1,1,1,0,0,0
114802,3. MED ONLY,0.224787,-0.492593,0.000000,-0.133220,-0.516252,0.000000,DUTCHESS COUNTY SELF-INS PLAN,0.212179,DUTCHESS,...,0.0,0.497909,2022.0,1,1,1,0,0,0,1
114803,4. TEMPORARY,1.471337,0.990741,1.387051,-1.511687,1.011472,0.029046,NEW JERSEY MANUFACTURERS,0.497548,NASSAU,...,0.0,1.497677,2020.0,1,1,1,1,0,0,1


In [15]:
train_df = train_df.drop(['Agreement Reached', 'Alternative Dispute Resolution_U', 'Zip Code'], axis=1, errors='ignore')
val_df = val_df.drop(['Agreement Reached', 'Alternative Dispute Resolution_U', 'Zip Code'], axis=1, errors='ignore')

In [16]:
# Get the column names of both DataFrames
train_cols = set(train_df.columns)
val_cols = set(val_df.columns)

# Find the differences
diff_train = train_cols - val_cols  # Columns in train_df but not in val_df
diff_val = val_cols - train_cols  # Columns in val_df but not in train_df

# Print the differences
if diff_train:
    print("Columns in train_df but not in val_df:")
    print(diff_train)

if diff_val:
    print("\nColumns in val_df but not in train_df:")
    print(diff_val)

if not diff_train and not diff_val:
    print("Both DataFrames have the same columns.")

Both DataFrames have the same columns.


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

test_df['Accident Year'] = scaler.fit_transform(test_df[['Accident Year']])
test_df['First Hearing Date'] = scaler.fit_transform(test_df[['First Hearing Date']])
val_df['Accident Year'] = scaler.fit_transform(val_df[['Accident Year']])
val_df['First Hearing Date'] = scaler.fit_transform(val_df[['First Hearing Date']])
train_df['Accident Year'] = scaler.fit_transform(train_df[['Accident Year']])
train_df['First Hearing Date'] = scaler.fit_transform(train_df[['First Hearing Date']])

In [9]:
categorical_cols = train_df.select_dtypes(include=['object']).columns
num_categorical_cols = len(categorical_cols)
print(f"Number of categorical columns: {num_categorical_cols}")
print(f"Categorical columns: {categorical_cols}")

Number of categorical columns: 4
Categorical columns: Index(['Claim Injury Type', 'Carrier Name', 'County of Injury',
       'District Name'],
      dtype='object')


## 1.1. The train-test split

8:2 splitting target column

In [17]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Prepare training data
X_train = train_df.drop('Claim Injury Type', axis=1)  # Independent variables
y_train = train_df['Claim Injury Type']  # Target variable

# Prepare validation/test data
X_test = val_df.drop('Claim Injury Type', axis=1)  # Independent variables
y_test = val_df['Claim Injury Type']  # Target variable

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Print the split ratio
total_samples = len(train_df) + len(val_df)
print('train:{}% | test:{}%'.format(
    round(len(train_df)/total_samples * 100, 2),
    round(len(val_df)/total_samples * 100, 2)
))

# Optional: Display all columns
pd.set_option('display.max_columns', None)

# Verify shapes
print("\nData shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

train:80.0% | test:20.0%

Data shapes:
X_train shape: (459220, 26)
X_test shape: (114805, 26)
y_train shape: (459220,)
y_test shape: (114805,)


In [18]:
X_train

,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M
0,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,PROPERTY AND CASUALTY,0.497548,QUEENS,NYC,-0.037533,1.098612,0.017597,0.463375,0.051126,0.063035,0.090122,1.497677,0.092321,1,1,1,1,0,0,0
1,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,HEALTH & HOSPITAL CORP.,0.212179,BRONX,NYC,-1.477658,0.000000,0.199081,0.463375,0.039230,0.010374,0.073124,-0.501860,0.632632,1,1,1,0,0,0,0
2,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,AMERICAN ZURICH INSURANCE CO,0.497548,KINGS,NYC,-0.037533,0.000000,0.029226,0.463375,0.005629,0.267488,0.032729,-0.001976,-0.447990,1,1,1,0,0,0,1
3,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,"NORDSTROM, INC.",0.091843,KINGS,NYC,-0.037533,0.000000,0.075944,0.463375,0.018845,0.192304,0.013745,-1.001744,0.092321,1,1,1,0,0,0,1
4,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,NEW YORK BLACK CAR OPERATORS',0.091843,NASSAU,NYC,-1.413414,0.000000,0.065062,0.463375,0.029073,0.097106,0.073124,0.497909,0.632632,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,-1.241742,-0.150000,1.409856,1.245247,-0.162524,-2.651452,INDEMNITY INSURANCE CO OF,0.497548,WESTCHESTER,NYC,0.396110,0.693147,0.199081,0.093731,0.007646,0.004915,0.014207,1.497677,0.092321,1,1,1,1,0,0,1
459216,0.884725,-0.009259,1.383011,-0.858729,-0.017208,-2.307054,REDWOOD FIRE & CASUALTY,0.497548,QUEENS,NYC,0.171258,1.945910,0.199081,0.463375,0.064603,0.192304,0.039561,-1.001744,0.092321,1,1,1,1,0,0,1
459217,1.031378,-0.292593,1.361083,-1.003831,-0.309751,0.000000,UTICA MUTUAL INS CO,0.497548,ERIE,BUFFALO,-0.037533,0.000000,0.023039,0.236296,0.020729,0.267488,0.083779,0.997793,0.092321,1,1,1,0,0,0,0
459218,-1.241742,0.642593,0.000000,1.172696,0.655832,0.000000,NEW HAMPSHIRE INSURANCE CO,0.497548,KINGS,NYC,-0.037533,0.000000,0.053783,0.463375,0.055488,0.267488,0.032729,-1.001744,-0.447990,1,1,1,0,0,0,1


Stratified Sampling: Ensures that the training and testing sets maintain the same class distribution as the original dataset, reducing bias.


Repeated Hold-Out: The process is repeated multiple times with different splits, and the average performance is considered.

In [19]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Identify categorical columns
categorical_features_indices = [i for i, dtype in enumerate(X_train.dtypes)
                              if dtype == 'object' or dtype == 'category']

# Create CatBoost Pools with categorical features
train_pool = Pool(data=X_train,
                 label=y_train,
                 cat_features=categorical_features_indices)

test_pool = Pool(data=X_test,
                label=y_test,
                cat_features=categorical_features_indices)

# Initialize model
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    verbose=0
)

# Fit using pools
model.fit(train_pool)

# Make predictions for both train and test sets
y_pred_train = model.predict(train_pool)
y_pred_test = model.predict(test_pool)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print(f'\nNumber of categorical features detected: {len(categorical_features_indices)}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

Training Metrics:
Accuracy Score: 0.7839
Macro F1 Score: 0.3579

Testing Metrics:
Accuracy Score: 0.7838
Macro F1 Score: 0.3622

Number of categorical features detected: 3

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.76      0.39      0.51      2495
           1       0.84      0.98      0.91     58216
           2       0.49      0.04      0.08     13781
           3       0.71      0.88      0.78     29701
           4       0.71      0.51      0.60      9656
           5       0.00      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       1.00      0.01      0.02        94

    accuracy                           0.78    114805
   macro avg       0.56      0.35      0.36    114805
weighted avg       0.74      0.78      0.73    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.0002
F1 Score Difference (Train - Test): -0.0043


In [20]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Save target variable before dropping categoricals
y_train = train_df['Claim Injury Type']  # Target variable
y_test = val_df['Claim Injury Type']  # Target variable

# Function to remove categorical columns
def drop_categorical_columns(df):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    return df.drop(columns=categorical_cols)

# Drop categorical columns from train and validation sets
X_train = drop_categorical_columns(train_df)
X_test = drop_categorical_columns(val_df)

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Print the split ratio
total_samples = len(train_df) + len(val_df)
print('train:{}% | test:{}%'.format(
    round(len(train_df)/total_samples * 100, 2),
    round(len(val_df)/total_samples * 100, 2)
))

# Optional: Display all columns
pd.set_option('display.max_columns', None)

# Verify shapes and remaining features
print("\nData shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Print information about removed features
original_features = train_df.shape[1] - 1  # Subtract 1 for target column
remaining_features = X_train.shape[1]
print(f"\nFeature reduction:")
print(f"Original number of features: {original_features}")
print(f"Remaining numeric features: {remaining_features}")
print(f"Number of categorical features removed: {original_features - remaining_features}")

train:80.0% | test:20.0%

Data shapes:
X_train shape: (459220, 23)
X_test shape: (114805, 23)
y_train shape: (459220,)
y_test shape: (114805,)

Feature reduction:
Original number of features: 26
Remaining numeric features: 23
Number of categorical features removed: 3


In [21]:
X_train

,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Type,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M
0,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,0.497548,-0.037533,1.098612,0.017597,0.463375,0.051126,0.063035,0.090122,1.497677,0.092321,1,1,1,1,0,0,0
1,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,0.212179,-1.477658,0.000000,0.199081,0.463375,0.039230,0.010374,0.073124,-0.501860,0.632632,1,1,1,0,0,0,0
2,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,0.497548,-0.037533,0.000000,0.029226,0.463375,0.005629,0.267488,0.032729,-0.001976,-0.447990,1,1,1,0,0,0,1
3,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,0.091843,-0.037533,0.000000,0.075944,0.463375,0.018845,0.192304,0.013745,-1.001744,0.092321,1,1,1,0,0,0,1
4,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,0.091843,-1.413414,0.000000,0.065062,0.463375,0.029073,0.097106,0.073124,0.497909,0.632632,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,-1.241742,-0.150000,1.409856,1.245247,-0.162524,-2.651452,0.497548,0.396110,0.693147,0.199081,0.093731,0.007646,0.004915,0.014207,1.497677,0.092321,1,1,1,1,0,0,1
459216,0.884725,-0.009259,1.383011,-0.858729,-0.017208,-2.307054,0.497548,0.171258,1.945910,0.199081,0.463375,0.064603,0.192304,0.039561,-1.001744,0.092321,1,1,1,1,0,0,1
459217,1.031378,-0.292593,1.361083,-1.003831,-0.309751,0.000000,0.497548,-0.037533,0.000000,0.023039,0.236296,0.020729,0.267488,0.083779,0.997793,0.092321,1,1,1,0,0,0,0
459218,-1.241742,0.642593,0.000000,1.172696,0.655832,0.000000,0.497548,-0.037533,0.000000,0.053783,0.463375,0.055488,0.267488,0.032729,-1.001744,-0.447990,1,1,1,0,0,0,1


In [33]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Convert to XGBoost's DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Enhanced parameters for multiclass classification
params = {
    'objective': 'multi:softmax',          # Multi-class classification
    'num_class': len(np.unique(y_train)),  # Number of classes in the target
    'learning_rate': 0.05,                 # Reduced learning rate for better convergence
    'max_depth': 6,                        # Reduced depth to prevent overfitting
    'min_child_weight': 3,                 # Increased to make splits more conservative
    'subsample': 0.8,                      # Slightly reduced to improve generalization
    'colsample_bytree': 0.8,               # Slightly reduced for better regularization
    'gamma': 0.2,                          # Minimum loss reduction required for a split
    'lambda': 2,                           # L2 regularization to penalize large weights
    'alpha': 1,                            # L1 regularization to encourage sparsity
    'eval_metric': ['mlogloss', 'merror']  # Evaluation metrics
}

model = xgb.train(
    params,
    dtrain,
    num_boost_round=300,                     # Increased rounds for slower, detailed learning
    evals=[(dtrain, 'train'), (dtest, 'validation')],
    early_stopping_rounds=30,                # Increased for more stability during training
    verbose_eval=50                          # Less frequent logs for cleaner output
)

# Rest of the code remains the same
y_pred_train = model.predict(dtrain)
y_pred_test = model.predict(dtest)

train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

importance = model.get_score(importance_type='gain')
importance_df = pd.DataFrame.from_dict(importance, orient='index', columns=['importance'])
importance_df = importance_df.sort_values('importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(importance_df)

[0]	train-mlogloss:1.97536	train-merror:0.25184	validation-mlogloss:1.97535	validation-merror:0.25210
[50]	train-mlogloss:0.74427	train-merror:0.21685	validation-mlogloss:0.74635	validation-merror:0.21760
[100]	train-mlogloss:0.62388	train-merror:0.21187	validation-mlogloss:0.62838	validation-merror:0.21320
[150]	train-mlogloss:0.59185	train-merror:0.20902	validation-mlogloss:0.59909	validation-merror:0.21094
[200]	train-mlogloss:0.57861	train-merror:0.20734	validation-mlogloss:0.58872	validation-merror:0.20985
[250]	train-mlogloss:0.57072	train-merror:0.20586	validation-mlogloss:0.58360	validation-merror:0.20922
[299]	train-mlogloss:0.56491	train-merror:0.20439	validation-mlogloss:0.58061	validation-merror:0.20851
Training Metrics:
Accuracy Score: 0.7956
Macro F1 Score: 0.4744

Testing Metrics:
Accuracy Score: 0.7915
Macro F1 Score: 0.4331

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.73      0.51      0.60   

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize Logistic Regression classifier
model = LogisticRegression(
    random_state=42,
    max_iter=1000,  # Added max iterations to avoid convergence warnings
    multi_class='auto'  # Automatically choose between 'ovr' and 'multinomial'
)

# Train model
model.fit(X_train, y_train)

# Make predictions for both train and test sets
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

Training Metrics:
Accuracy Score: 0.7534
Macro F1 Score: 0.2666

Testing Metrics:
Accuracy Score: 0.7538
Macro F1 Score: 0.2653

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.27      0.01      0.02      2495
           1       0.82      0.99      0.90     58216
           2       0.31      0.04      0.07     13781
           3       0.67      0.88      0.76     29701
           4       0.61      0.24      0.35      9656
           5       0.00      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       1.00      0.01      0.02        94

    accuracy                           0.75    114805
   macro avg       0.46      0.27      0.27    114805
weighted avg       0.69      0.75      0.69    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): -0.0004
F1 Score Difference (Train - Test): 0.0014


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize KNN classifier with basic parameters
model = KNeighborsClassifier(
    n_neighbors=5,  # Default value, can be tuned
    weights='uniform',
    metric='minkowski',
    p=2  # Euclidean distance
)

# Train model
model.fit(X_train, y_train)

# Make predictions for both train and test sets
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

# Model specific information
print("\nModel Information:")
print(f"Number of Neighbors Used: {model.n_neighbors}")
print(f"Distance Metric: {model.metric}")
print(f"Number of Classes: {len(np.unique(y_test))}")

Training Metrics:
Accuracy Score: 0.7992
Macro F1 Score: 0.4326

Testing Metrics:
Accuracy Score: 0.7453
Macro F1 Score: 0.3315

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.47      0.29      0.36      2495
           1       0.84      0.96      0.89     58216
           2       0.27      0.10      0.15     13781
           3       0.69      0.79      0.74     29701
           4       0.56      0.41      0.48      9656
           5       0.13      0.01      0.02       842
           6       0.00      0.00      0.00        20
           7       0.17      0.01      0.02        94

    accuracy                           0.75    114805
   macro avg       0.39      0.32      0.33    114805
weighted avg       0.69      0.75      0.71    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.0539
F1 Score Difference (Train - Test): 0.1010

Model Information:
Number of Neighbors Used: 5
Dist

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize Random Forest classifier with basic parameters
model = RandomForestClassifier(
    n_estimators=1000,          # More trees for better performance
    max_depth=30,               # Limit tree depth to avoid overfitting
    min_samples_split=10,       # Minimum samples required to split a node
    min_samples_leaf=5,         # Minimum samples required to be at a leaf node
    max_features='sqrt',        # Use the square root of features for splits
    random_state=42,            # Ensure reproducibility
    n_jobs=-1,                  # Use all available cores for faster computation
    bootstrap=True,             # Use bootstrap sampling to introduce randomness
    oob_score=True,             # Enable out-of-bag estimation for validation
    class_weight='balanced'     # Handle imbalanced classes if needed
)
# Train model
model.fit(X_train, y_train)

# Make predictions for both train and test sets
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

# Feature importance (top 10)
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
})
importance_df = importance_df.sort_values('importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(importance_df)

# Model specific information
print("\nModel Information:")
print(f"Number of Trees: {model.n_estimators}")
print(f"Max Depth: {model.max_depth}")
print(f"Number of Features Used: {model.n_features_in_}")

Training Metrics:
Accuracy Score: 0.8901
Macro F1 Score: 0.8593

Testing Metrics:
Accuracy Score: 0.7736
Macro F1 Score: 0.4475

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.54      0.64      0.58      2495
           1       0.87      0.94      0.90     58216
           2       0.38      0.15      0.21     13781
           3       0.76      0.79      0.77     29701
           4       0.58      0.72      0.64      9656
           5       0.19      0.06      0.09       842
           6       0.00      0.00      0.00        20
           7       0.33      0.45      0.38        94

    accuracy                           0.77    114805
   macro avg       0.45      0.47      0.45    114805
weighted avg       0.74      0.77      0.75    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.1165
F1 Score Difference (Train - Test): 0.4118

Top 10 Most Important Features:
                   